# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
pip install evaluate scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Section imports
from datasets import load_dataset
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
import evaluate

# Create variables and constants
FOUNDATION_MODEL = "gpt2"
tokenized_dataset = {}
splits = ["train", "test"]

# Downloading the date set
column_label = "verse_text"
dataset_name='poem_sentiment'
dataset = load_dataset(dataset_name, split='train').train_test_split(test_size=0.2, shuffle=True, seed=24)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

c:\Users\freda\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Defined function to process tokenized inputs

def tokenized_inputs(inputs, tokenizer):
    prompt = inputs[column_label]
    tokenizer.truncation_side = "left"
    tokenized_items = tokenizer(
        prompt,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    return tokenized_items



# Defined function to evaluate accuracy of model
# https://huggingface.co/spaces/evaluate-metric/precision
def compute_metrics(eval_preds):
    metric = evaluate.load("precision")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="micro")


# Created Generic function to return the Trainer object
def get_trainer(trainer_model, dataset_name, metrics, num_epoch, output_dir, tokenizer):
    trained_model = Trainer(
        model=trainer_model,
        args=TrainingArguments(
            output_dir=output_dir,
            learning_rate=2e-5,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=96,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            num_train_epochs=num_epoch,
            weight_decay=0.01,
            load_best_model_at_end=True
        ),
        train_dataset=dataset_name["train"],
        eval_dataset=dataset_name["test"],
        tokenizer=tokenizer,
        compute_metrics=metrics,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    )
    return trained_model


# Define Tokenizer function
def get_tokenizer(modal_name):
    tokenizer = AutoTokenizer.from_pretrained(modal_name)
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

In [4]:
# Configuring the tokenizer and adding padding tokens function
tokenizer_model = get_tokenizer(FOUNDATION_MODEL)

# Tokenize inputs
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenized_inputs(x, tokenizer_model), batched=True
    )

print("tokenized_dataset", tokenized_dataset["train"])

tokenized_dataset Dataset({
    features: ['id', 'verse_text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 713
})


In [5]:
# Create Label objects to pass down to ouw model
id2label = {0: "negative", 1: "positive", 2: "no impact", 3: "mixed"}
label2id ={"negative": 0, "positive": 1, "no impact": 2, "mixed": 3}

# Loading the foundation model with the correct number of labels and identifiers
model = AutoModelForSequenceClassification.from_pretrained(
    FOUNDATION_MODEL,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)

model.config.pad_token_id = model.config.eos_token_id
model.resize_token_embeddings(len(tokenizer_model))
model.to(device)

# Created Trainer Object and loaded the original model
gpt2_model_trainer = get_trainer(
    trainer_model=model,
    dataset_name=tokenized_dataset,
    metrics=compute_metrics,
    num_epoch=1,
    output_dir="./",
    tokenizer=tokenizer_model
)
print(f'Evaluating foundation model...')
original_model_eval = gpt2_model_trainer.evaluate()
print(original_model_eval)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\freda\miniconda3\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Evaluating foundation model...


100%|██████████| 2/2 [00:02<00:00,  1.03s/it]

{'eval_loss': 10.239521026611328, 'eval_precision': 0.20670391061452514, 'eval_runtime': 3.8872, 'eval_samples_per_second': 46.049, 'eval_steps_per_second': 0.515}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [6]:
from peft import LoraConfig, get_peft_model, TaskType

In [7]:
# Configure LoRa settings
targets = ["c_proj", "c_fc", "c_attn"]
config = LoraConfig(
    r=1,
    lora_alpha=8,
    lora_dropout=0.01,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    target_modules=targets,
    fan_in_fan_out=True
)
# Create Peft Model
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 150,528 || all params: 124,593,408 || trainable%: 0.1208


In [8]:
# Freeze All parameters except those being fine tunes
for name, param in lora_model.named_parameters():
    if any(target in name for target in targets):
        param.requires_grad = True
    else:
        param.requires_grad = False

In [9]:
# Define Train Object Using Lora Model from Peft
lora_trainer = get_trainer(
    trainer_model=lora_model, 
    dataset_name=tokenized_dataset, 
    metrics=compute_metrics, 
    num_epoch=1, 
    output_dir="./gpt-lora",
    tokenizer=tokenizer_model
)
lora_trainer.train()
# Print lora_trainer evaluation
eval_lora = lora_trainer.evaluate()
print(eval_lora)

c:\Users\freda\miniconda3\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 23/23 [00:05<00:00,  5.13it/s]c:\Users\freda\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'eval_loss': 2.817417621612549, 'eval_precision': 0.25139664804469275, 'eval_runtime': 1.899, 'eval_samples_per_second': 94.26, 'eval_steps_per_second': 1.053, 'epoch': 1.0}


100%|██████████| 23/23 [00:07<00:00,  3.11it/s]


{'train_runtime': 7.4061, 'train_samples_per_second': 96.272, 'train_steps_per_second': 3.106, 'train_loss': 5.19460097603176, 'epoch': 1.0}


100%|██████████| 2/2 [00:01<00:00,  1.13it/s]

{'eval_loss': 2.817417621612549, 'eval_precision': 0.25139664804469275, 'eval_runtime': 2.0208, 'eval_samples_per_second': 88.58, 'eval_steps_per_second': 0.99, 'epoch': 1.0}


In [10]:
lora_model.save_pretrained("gpt-lora-trained")

c:\Users\freda\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [11]:
from peft import AutoPeftModelForSequenceClassification
import torch

In [12]:
new_lora_tokenizer = get_tokenizer("./gpt-lora/checkpoint-23")
# Loading saved PEFT model from directory
lora_peft_model = AutoPeftModelForSequenceClassification.from_pretrained("./gpt-lora-trained", num_labels=4)
lora_peft_model.config.pad_token_id = lora_peft_model.config.eos_token_id
lora_peft_model.resize_token_embeddings(len(tokenizer_model))
lora_peft_model.to(device)

# Load Trained Tokenizer
lora_model_trainer = get_trainer(
    trainer_model=lora_peft_model,
    dataset_name=tokenized_dataset,
    metrics=compute_metrics,
    num_epoch=1,
    output_dir="./",
    tokenizer=tokenizer_model
)
lora_model_eval = lora_model_trainer.evaluate()
print(lora_model_eval)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\freda\miniconda3\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 2/2 [00:02<00:00,  1.05s/it]

{'eval_loss': 10.192325592041016, 'eval_precision': 0.20670391061452514, 'eval_runtime': 2.3547, 'eval_samples_per_second': 76.017, 'eval_steps_per_second': 0.849}


In [13]:
# Starting Inference
input_text = "in prosperous days. like a dim, waning lamp"
model_inputs = new_lora_tokenizer(input_text, return_tensors="pt").to(device)

In [14]:
with torch.no_grad():
    outputs = lora_peft_model(**model_inputs)
    results = torch.argmax(outputs.logits, dim=-1)
# Converting Prediction to Label
predicted_label = id2label[results.item()]

In [15]:
# Input Text with Predicted outout
print(f"Input: {input_text}\n Predicted label: {predicted_label}")

Input: in prosperous days. like a dim, waning lamp
 Predicted label: negative


In [16]:
# comparing Evaluation Results
print(f'Original model eval: {original_model_eval} \nLora Model Eval: {lora_model_eval}')

Original model eval: {'eval_loss': 10.239521026611328, 'eval_precision': 0.20670391061452514, 'eval_runtime': 3.8872, 'eval_samples_per_second': 46.049, 'eval_steps_per_second': 0.515} 
Lora Model Eval: {'eval_loss': 10.192325592041016, 'eval_precision': 0.20670391061452514, 'eval_runtime': 2.3547, 'eval_samples_per_second': 76.017, 'eval_steps_per_second': 0.849}
